In [1]:
import requests
import pandas as pd
import numpy as np
import json
import re
from bs4 import BeautifulSoup

In [2]:
# поиск по запросу "отозвана"
link = 'https://www.cbr.ru/press/month_archive/press_archive/?UniDbQuery.Posted=True&UniDbQuery.Phrase=%D0%BE%D1%82%D0%BE%D0%B7%D0%B2%D0%B0%D0%BD%D0%B0&UniDbQuery.Search=0&UniDbQuery.FromDate=01.01.2020&UniDbQuery.ToDate=21.02.2020'
page = requests.get(link).content
soup1=BeautifulSoup(page, 'html.parser')

In [3]:
#объекты с основной информацией и датой
urls_inf = soup1.find_all('div', attrs = {'class': 'link'})
urls_date = soup1.find_all('div', attrs = {'class': 'source'})

### Тестовые списки

In [4]:
#Название пресс-релиза
title_press = []
for i in urls_inf:
    test = i.getText(separator=u'/').strip()
    title_press.append(test)
title_press

['Отозвана лицензия на осуществление банковских операций у АКБ «АПАБАНК» (АО)',
 'Отозвана лицензия на осуществление банковских операций у АО «НВКбанк»',
 'Отозвана лицензия на осуществление банковских операций у ПАО КБ «ПФС-БАНК»',
 'Отозвана лицензия на осуществление банковских операций у ООО КБ «Нэклис-Банк»']

In [5]:
#Название банка
title = []
for i in title_press:
    l = i.find(' у ') + 3
    title.append(i[l:])
title
    

['АКБ «АПАБАНК» (АО)',
 'АО «НВКбанк»',
 'ПАО КБ «ПФС-БАНК»',
 'ООО КБ «Нэклис-Банк»']

In [6]:
#дата пресс-релиза
date = []
for i in urls_date:
    test = i.getText(separator=u'/').strip()
    date.append(test)
date

['31.01.2020', '24.01.2020', '17.01.2020', '10.01.2020']

In [7]:
#Ссылка на пресс-релиз
lin_fin = []
for a in urls_inf:
    z = a.find('a', href=True)
    lin_fin.append('https://www.cbr.ru/press' + z['href'][5:])
lin_fin    

['https://www.cbr.ru/press/PR/?file=31012020_083642ik2020-01-31T08_36_04.htm',
 'https://www.cbr.ru/press/PR/?file=24012020_074026ik2020-01-24T07_39_24.htm',
 'https://www.cbr.ru/press/PR/?file=17012020_083449ik2020-01-17T08_33_40.htm',
 'https://www.cbr.ru/press/PR/?file=10012020_083823ik2020-01-10T08_37_09.htm']

### Серьезные дела

In [8]:
months=list(range(1,12))
years=list(range(2006,2020))

In [9]:
inf_sum = []
date_sum = []

for i in range(2006, 2021):
    for k in range(1, 11, 3):
        if any( [k == 4, k == 7]):
            link = 'https://www.cbr.ru/press/month_archive/press_archive/?UniDbQuery.Posted=True&UniDbQuery.' + 'Phrase=%D0%BE%D1%82%D0%BE%D0%B7%D0%B2%D0%B0%D0%BD%D0%B0&UniDbQuery.Search=0&UniDbQuery.FromDate=01.' + str(k) + '.' + str(i) + '&UniDbQuery.ToDate=30.' + str(k+2) + '.' + str(i)
            page = requests.get(link).content
            soup1=BeautifulSoup(page, 'html.parser')
            urls_inf = soup1.find_all('div', attrs = {'class': 'link'})
            urls_date = soup1.find_all('div', attrs = {'class': 'source'})
            inf_sum = inf_sum + urls_inf
            date_sum = date_sum + urls_date
        else:
            link = 'https://www.cbr.ru/press/month_archive/press_archive/?UniDbQuery.Posted=True&UniDbQuery.' + 'Phrase=%D0%BE%D1%82%D0%BE%D0%B7%D0%B2%D0%B0%D0%BD%D0%B0&UniDbQuery.Search=0&UniDbQuery.FromDate=01.' + str(k) + '.' + str(i) + '&UniDbQuery.ToDate=31.' + str(k+2) + '.' + str(i)
            page = requests.get(link).content
            soup1=BeautifulSoup(page, 'html.parser')
            urls_inf = soup1.find_all('div', attrs = {'class': 'link'})
            urls_date = soup1.find_all('div', attrs = {'class': 'source'})
            inf_sum = inf_sum + urls_inf
            date_sum = date_sum + urls_date
            

In [10]:
#Название пресс-релиза
title_press1 = []
for i in inf_sum:
    test = i.getText(separator=u'/').strip()
    title_press1.append(test)

In [11]:
#дата пресс-релиза
date1 = []
for i in date_sum:
    test = i.getText(separator=u'/').strip()
    date1.append(test)

In [12]:
#Ссылка на пресс-релиз
lin_fin1 = []
for a in inf_sum:
    z = a.find('a', href=True)
    lin_fin1.append('https://www.cbr.ru/press' + z['href'][5:])

In [13]:
#Создание таблицы
dict1 = {'Title': title_press1,
         'Date': date1,
         'Link': lin_fin1}
df=pd.DataFrame(dict1)
df


,Title,Date,Link
0,Об отзыве лицензии и назначении временной адми...,30.03.2006,https://www.cbr.ru/press/PR/?file=060330_09334...
1,Об отзыве лицензии на осуществление банковских...,26.01.2006,https://www.cbr.ru/press/PR/?file=060126_09213...
2,Об отзыве лицензии и назначении временной адми...,30.03.2006,https://www.cbr.ru/press/PR/?file=060330_09334...
3,Об отзыве лицензии и назначении временной адми...,23.03.2006,https://www.cbr.ru/press/PR/?file=060323_09310...
4,Об отзыве лицензии на осуществление банковских...,27.02.2006,https://www.cbr.ru/press/PR/?file=060227_09343...
...,...,...,...
723,Отозвана лицензия на осуществление банковских ...,01.11.2019,https://www.cbr.ru/press/PR/?file=01112019_054...
724,Отозвана лицензия на осуществление банковских ...,31.01.2020,https://www.cbr.ru/press/PR/?file=31012020_083...
725,Отозвана лицензия на осуществление банковских ...,24.01.2020,https://www.cbr.ru/press/PR/?file=24012020_074...
726,Отозвана лицензия на осуществление банковских ...,17.01.2020,https://www.cbr.ru/press/PR/?file=17012020_083...


In [14]:
#Проверка на дупликаты
len(df.Link.unique())

728

In [15]:
##### Пробую проделать с запросом отзыв
inf_sum2 = []
date_sum2 = []

for i in range(2006, 2021):
    for k in range(1, 11, 3):
        if any( [k == 4, k == 7]):
            link = 'https://www.cbr.ru/press/month_archive/press_archive/?UniDbQuery.Posted=True&UniDbQuery.Phrase=%D0%BE%D1%82%D0%B7%D1%8B%D0%B2&UniDbQuery.Search=0&UniDbQuery.FromDate=01.' + str(k) + '.' + str(i) + '&UniDbQuery.ToDate=30.' + str(k+2) + '.' + str(i)
            page = requests.get(link).content
            soup2=BeautifulSoup(page, 'html.parser')
            urls_inf2 = soup2.find_all('div', attrs = {'class': 'link'})
            urls_date2 = soup2.find_all('div', attrs = {'class': 'source'})
            inf_sum2 = inf_sum2 + urls_inf2
            date_sum2 = date_sum2 + urls_date2
        else:
            link = 'https://www.cbr.ru/press/month_archive/press_archive/?UniDbQuery.Posted=True&UniDbQuery.Phrase=%D0%BE%D1%82%D0%B7%D1%8B%D0%B2&UniDbQuery.Search=0&UniDbQuery.FromDate=01.' + str(k) + '.' + str(i) + '&UniDbQuery.ToDate=31.' + str(k+2) + '.' + str(i)
            page = requests.get(link).content
            soup2=BeautifulSoup(page, 'html.parser')
            urls_inf2 = soup2.find_all('div', attrs = {'class': 'link'})
            urls_date2 = soup2.find_all('div', attrs = {'class': 'source'})
            inf_sum2 = inf_sum2 + urls_inf2
            date_sum2 = date_sum2 + urls_date2

In [16]:
#Название пресс-релиза
title_press2 = []
for i in inf_sum2:
    test2 = i.getText(separator=u'/').strip()
    title_press2.append(test2)

In [17]:
#дата пресс-релиза
date2 = []
for i in date_sum2:
    test2 = i.getText(separator=u'/').strip()
    date2.append(test2)

In [18]:
#Ссылка на пресс-релиз
lin_fin2 = []
for a in inf_sum2:
    z2 = a.find('a', href=True)
    lin_fin2.append('https://www.cbr.ru/press' + z2['href'][5:])

In [19]:
#Создание таблицы
dict2 = {'Title': title_press2,
         'Date': date2,
         'Link': lin_fin2}
df2=pd.DataFrame(dict2)
df2

,Title,Date,Link
0,Об отзыве лицензии на осуществление банковских...,26.01.2006,https://www.cbr.ru/press/PR/?file=060126_09213...
1,Об отзыве лицензии на осуществление банковских...,04.05.2006,https://www.cbr.ru/press/PR/?file=060504_09310...
2,Об отзыве лицензии на осуществление банковских...,24.08.2006,https://www.cbr.ru/press/PR/?file=060824_09330...
3,Об отзыве лицензии на осуществление банковских...,17.07.2006,https://www.cbr.ru/press/PR/?file=060717_11451...
4,Об отзыве лицензии на осуществление банковских...,10.08.2006,https://www.cbr.ru/press/PR/?file=060810_10112...
...,...,...,...
448,Об отзыве у банка «РТС-Банк» лицензии на осуще...,14.03.2019,https://www.cbr.ru/press/PR/?file=14032019_074...
449,Об отзыве у банка «РАДИОТЕХБАНК» лицензии на о...,31.01.2019,https://www.cbr.ru/press/PR/?file=31012019_083...
450,Об отзыве у банка «Еврокапитал-Альянс» лицензи...,25.01.2019,https://www.cbr.ru/press/PR/?file=25012019_083...
451,Об отзыве у банка «РОСКОМСНАББАНК» лицензии на...,07.03.2019,https://www.cbr.ru/press/PR/?file=07032019_065...


In [20]:
len(df2.Link.unique())

453

In [21]:
### Сливаю таблички
data_new = pd.concat([df, df2] , axis = 0)

In [22]:
data_new

,Title,Date,Link
0,Об отзыве лицензии и назначении временной адми...,30.03.2006,https://www.cbr.ru/press/PR/?file=060330_09334...
1,Об отзыве лицензии на осуществление банковских...,26.01.2006,https://www.cbr.ru/press/PR/?file=060126_09213...
2,Об отзыве лицензии и назначении временной адми...,30.03.2006,https://www.cbr.ru/press/PR/?file=060330_09334...
3,Об отзыве лицензии и назначении временной адми...,23.03.2006,https://www.cbr.ru/press/PR/?file=060323_09310...
4,Об отзыве лицензии на осуществление банковских...,27.02.2006,https://www.cbr.ru/press/PR/?file=060227_09343...
...,...,...,...
448,Об отзыве у банка «РТС-Банк» лицензии на осуще...,14.03.2019,https://www.cbr.ru/press/PR/?file=14032019_074...
449,Об отзыве у банка «РАДИОТЕХБАНК» лицензии на о...,31.01.2019,https://www.cbr.ru/press/PR/?file=31012019_083...
450,Об отзыве у банка «Еврокапитал-Альянс» лицензи...,25.01.2019,https://www.cbr.ru/press/PR/?file=25012019_083...
451,Об отзыве у банка «РОСКОМСНАББАНК» лицензии на...,07.03.2019,https://www.cbr.ru/press/PR/?file=07032019_065...


In [23]:
len(data_new.Link.unique())

732

In [24]:
data_new.drop_duplicates(subset ="Link", 
                     keep = 'first', inplace = True) 
#data_new.iloc[:,0]
df = data_new.reset_index(drop=True)
df

,Title,Date,Link
0,Об отзыве лицензии и назначении временной адми...,30.03.2006,https://www.cbr.ru/press/PR/?file=060330_09334...
1,Об отзыве лицензии на осуществление банковских...,26.01.2006,https://www.cbr.ru/press/PR/?file=060126_09213...
2,Об отзыве лицензии и назначении временной адми...,30.03.2006,https://www.cbr.ru/press/PR/?file=060330_09334...
3,Об отзыве лицензии и назначении временной адми...,23.03.2006,https://www.cbr.ru/press/PR/?file=060323_09310...
4,Об отзыве лицензии на осуществление банковских...,27.02.2006,https://www.cbr.ru/press/PR/?file=060227_09343...
...,...,...,...
727,Отозвана лицензия на осуществление банковских ...,10.01.2020,https://www.cbr.ru/press/PR/?file=10012020_083...
728,Об отзыве лицензии на осуществление банковских...,17.07.2006,https://www.cbr.ru/press/PR/?file=060717_11451...
729,Об Указании Банка России от 17 сентября 2009 г...,06.11.2009,https://www.cbr.ru/press/PR/?file=091106_16533...
730,Об информации Фонда гарантирования вкладов физ...,11.04.2013,https://www.cbr.ru/press/PR/?file=130411_18530...


In [39]:
df['Link'][0]

'https://www.cbr.ru/press/PR/?file=060330_093346bank1.htm'

### Вытаскиваем текст (тест)

In [25]:
dop = 'https://www.cbr.ru/press/PR/?file=31012020_083642ik2020-01-31T08_36_04.htm'
krot = requests.get(dop).content.decode('utf-8')
enot=BeautifulSoup(krot, 'html.parser')

In [35]:
nuzhno = enot.find('div', attrs = {'class': 'paper_layout'})
nuzhno1 = nuzhno.find_all(['p', 'ul'], recursive = False)

nuzhno1

[<p>
 <p>Банк России приказом от 31.01.2020 № ОД-166 отозвал лицензию на осуществление банковских операций у Аграрного профсоюзного акционерного коммерческого банка «АПАБАНК» (Акционерное общество) АКБ «АПАБАНК» (АО) (рег. № 2404, г. Москва, далее — АПАБАНК). По величине активов кредитная организация занимала 369 место в банковской системе Российской Федерации<sup>1</sup>. Банк не является участником системы страхования вкладов.</p>
 <p>Банк России принял такое решение в соответствии с п. 6 части первой ст. 20 Федерального закона «О банках и банковской деятельности»<sup>2</sup>, руководствуясь тем, что АПАБАНК:</p>
 <ul>
 <ul>
 <li>занижал величину необходимых к формированию резервов, а также завышал стоимость имущества в целях улучшения финансовых показателей и сокрытия своего реального финансового положения. Отражение в финансовой отчетности принимаемых кредитных рисков и реальной стоимости учтенной на балансе недвижимости по требованию Банка России выявило снижение (более 40%) капит

In [36]:
fom = []
for i in nuzhno1:
    i.text#.strip
    fom.append(re.sub('<[^>]*>', '', str(i)).replace('\xa0', ' ').split('\n'))
flatList = [ item for elem in fom for item in elem]

msg = '\n\n'.join([str(i) for i in flatList])
msg

### Вытаскиваем финально

In [47]:
#Вытащил тексты релизов
bob = []
for i in df['Link']:
    fom1 = []
    krot1 = requests.get(i).content.decode('utf-8')
    enot1=BeautifulSoup(krot1, 'html.parser')
    kot = enot1.find('div', attrs = {'class': 'paper_layout'})
    kot1 = kot.find_all(['p', 'ul'], recursive = False)
    for k in kot1:
        k.text#.strip
        fom1.append(re.sub('<[^>]*>', '', str(k)).replace('\xa0', ' ').split('\n'))
    flatList1 = [ item for elem in fom1 for item in elem]
    msg1 = str('\n\n'.join([str(i) for i in flatList1]))
    bob.append(msg1)    

In [142]:
#Вытащил причины где были
bob2 = []
for i in df['Link']:
    fom2 = []
    krot2 = requests.get(i).content.decode('utf-8')
    enot2=BeautifulSoup(krot2, 'html.parser')
    #kot2 = enot2.find('div', attrs = {'class': 'paper_layout'})
    kot21 = enot2.find_all('ul')#, recursive = False)
    for k in kot21:
        k.text#.strip
        fom2.append(re.sub('<[^>]*>', '', str(k)).replace('\xa0', ' ').split('\n'))
    flatList2 = [ item for elem in fom2 for item in elem]
    if len(flatList)>26:
        bor = flatList2[11:-15]
        bor1 = bor[len(bor)//2:]
        msg2 = str('\n\n'.join([str(i) for i in bor1]))
        bob2.append(msg2)        
    else:
        msg2 = str('\n\n'.join([str(i) for i in bor1]))
        bob2.append(msg2)

In [143]:
#Проверка на длину датасетов
print(len(bob))
print(len(bob2))

732
732


In [144]:
#Все соединил
Text = pd.Series(bob)
Reason = pd.Series(bob2)
df1 = pd.concat([df, Text, Reason], axis = 1)
df1.columns = ['Title', 'Date', 'Link', 'Text', 'Reason']
df1

,Title,Date,Link,Text,Reason
0,Об отзыве лицензии и назначении временной адми...,30.03.2006,https://www.cbr.ru/press/PR/?file=060330_09334...,Департамент внешних и общественных связей Банк...,
1,Об отзыве лицензии на осуществление банковских...,26.01.2006,https://www.cbr.ru/press/PR/?file=060126_09213...,Департамент внешних и общественных связей Банк...,
2,Об отзыве лицензии и назначении временной адми...,30.03.2006,https://www.cbr.ru/press/PR/?file=060330_09334...,Департамент внешних и общественных связей Банк...,
3,Об отзыве лицензии и назначении временной адми...,23.03.2006,https://www.cbr.ru/press/PR/?file=060323_09310...,Департамент внешних и общественных связей Банк...,
4,Об отзыве лицензии на осуществление банковских...,27.02.2006,https://www.cbr.ru/press/PR/?file=060227_09343...,Департамент внешних и общественных связей Банк...,
...,...,...,...,...,...
727,Отозвана лицензия на осуществление банковских ...,10.01.2020,https://www.cbr.ru/press/PR/?file=10012020_083...,Банк России приказом от 10.01.2020 № ОД-6 отоз...,полностью утратил собственные средства вследст...
728,Об отзыве лицензии на осуществление банковских...,17.07.2006,https://www.cbr.ru/press/PR/?file=060717_11451...,Департамент внешних и общественных связей Банк...,
729,Об Указании Банка России от 17 сентября 2009 г...,06.11.2009,https://www.cbr.ru/press/PR/?file=091106_16533...,Департамент внешних и общественных связей Банк...,
730,Об информации Фонда гарантирования вкладов физ...,11.04.2013,https://www.cbr.ru/press/PR/?file=130411_18530...,Департамент внешних и общественных связей Банк...,


In [145]:
#Проверил в каких релизах причины были выделены отдельно
df1['prior_cleansing']= df1['Title'].str.find('Отозван')
df2 = df1[df1.prior_cleansing >= 0]
df2

,Title,Date,Link,Text,Reason,prior_cleansing
713,Отозвана лицензия на осуществление банковских ...,12.09.2019,https://www.cbr.ru/press/PR/?file=12092019_034...,\n\nБанк России приказом от 12.09.2019 № ОД-20...,допустил уменьшение величины собственных средс...,0
714,Отозвана лицензия на осуществление банковских ...,29.07.2019,https://www.cbr.ru/press/PR/?file=29072019_083...,\n\nБанк России приказом от 29.07.2019 № ОД-1...,признал отсутствие отраженных в учете и отчетн...,0
715,Отозвана лицензия на осуществление банковских ...,26.07.2019,https://www.cbr.ru/press/PR/?file=26072019_083...,\n\nБанк России приказом от 26.07.2019 № ОД-1...,занижал размер резервов на возможные потери по...,0
716,Отозвана лицензия на осуществление банковских ...,13.12.2019,https://www.cbr.ru/press/PR/?file=13122019_084...,\n\nБанк России приказом от 13.12.2019 № ОД-28...,допускал нарушения нормативных актов Банка Рос...,0
717,Отозвана лицензия на осуществление банковских ...,13.12.2019,https://www.cbr.ru/press/PR/?file=13122019_083...,\n\nБанк России приказом от 13.12.2019 № ОД-28...,полностью утратил собственные средства (капита...,0
718,Отозвана лицензия на осуществление банковских ...,29.11.2019,https://www.cbr.ru/press/PR/?file=29112019_083...,\n\nБанк России приказом от 29.11.2019 № ОД-27...,допускал нарушения законодательства в области ...,0
719,Отозвана лицензия на осуществление банковских ...,22.11.2019,https://www.cbr.ru/press/PR/?file=22112019_084...,\n\n\n\nБанк России приказом от 22.11.2019 № О...,допускала нарушения законодательства в области...,0
720,Отозвана лицензия на осуществление банковских ...,22.11.2019,https://www.cbr.ru/press/PR/?file=22112019_083...,\n\nБанк России приказом от 22.11.2019 № ОД-26...,допускал нарушения законодательства в области ...,0
721,Отозвана лицензия на осуществление банковских ...,15.11.2019,https://www.cbr.ru/press/PR/?file=15112019_083...,Банк России приказом от 15.11.2019 № ОД-2609 о...,допускал нарушения законодательства и норматив...,0
722,Отозвана лицензия на осуществление банковских ...,07.11.2019,https://www.cbr.ru/press/PR/?file=07112019_083...,Банк России приказом от 07.11.2019 № ОД-2565 о...,допускал нарушения законодательства и норматив...,0


In [147]:
#Проверки
a = df1['Title'].str.find('Отозвана')
sum(map(lambda x : x>=0, a))

15

In [146]:
len(df1['Reason'].unique())

16

### Записал датасет

In [157]:
df1.to_excel("Banks.xlsx", index = False, encoding = 'utf-8-sig')

### Random check

In [149]:
print(df1['Text'][721])

Банк России приказом от 15.11.2019 № ОД-2609 отозвал лицензию на осуществление банковских операций у Общества с ограниченной ответственностью Коммерческий банк «Кредитинвест» ООО КБ «Кредитинвест» (рег. № 1197, Республика Дагестан, г. Кизилюрт, далее — Банк Кредитинвест). По величине активов кредитная организация занимала 386 место в банковской системе Российской Федерации1. 

Банк России принял такое решение в соответствии с п.п. 6 и 6.1 части первой ст. 20 Федерального закона «О банках и банковской деятельности»2, руководствуясь тем, что Банк Кредитинвест:





допускал нарушения законодательства и нормативных актов Банка России в области противодействия легализации (отмыванию) доходов, полученных преступным путем, и финансированию терроризма. Кредитная организация представляла в уполномоченный орган неполную и недостоверную информацию, в том числе по операциям, подлежащим обязательному контролю;

проводил сомнительные трансграничные операции;

нарушал федеральные законы, регулирующи

In [150]:
print(df1['Link'][721])

https://www.cbr.ru/press/PR/?file=15112019_083613ik2019-11-15T08_34_25.htm


In [148]:
print(df2['Reason'][721])

допускал нарушения законодательства и нормативных актов Банка России в области противодействия легализации (отмыванию) доходов, полученных преступным путем, и финансированию терроризма. Кредитная организация представляла в уполномоченный орган неполную и недостоверную информацию, в том числе по операциям, подлежащим обязательному контролю;

проводил сомнительные трансграничные операции;

нарушал федеральные законы, регулирующие банковскую деятельность, а также нормативные акты Банка России, в связи с чем регулятор в течение последних 12 месяцев неоднократно применял к нему меры.




